In [1]:
%matplotlib notebook

In [2]:
# Import dependencies

import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np

In [3]:
pathHR = 'HR_comma_sep.csv'
df_HR = pd.read_csv(pathHR)
df_HR

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14994,0.40,0.57,2,151,3,0,1,0,support,low
14995,0.37,0.48,2,160,3,0,1,0,support,low
14996,0.37,0.53,2,143,3,0,1,0,support,low
14997,0.11,0.96,6,280,4,0,1,0,support,low


Data Cleaning

In [4]:
df_HR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   sales                  14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [5]:
# Note dataset has no specific ID variable such as employee ID.
# Check frequency for different variables

df_HR['left'].value_counts()

0    11428
1     3571
Name: left, dtype: int64

In [6]:
df_HR['sales'].value_counts()

sales          4140
technical      2720
support        2229
IT             1227
product_mng     902
marketing       858
RandD           787
accounting      767
hr              739
management      630
Name: sales, dtype: int64

In [7]:
df_HR.rename(columns={'sales':'Departments'}, inplace = True)
df_HR.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Departments,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [8]:
# Align coding for variables with those from EmployeeAttrition.csv (and related dfs)
## In EmployeeAttrition.csv: 1 = Stayers, 2 = Leavers
## In current dataset: 1 = Left; 0 = Stay. Code below amends any deviation.

df_HR['left'] = df_HR['left'].replace({1: 2})
df_HR['left'] = df_HR['left'].replace({0: 1})
df_HR['left'].value_counts()

1    11428
2     3571
Name: left, dtype: int64

In [11]:
# Descriptive Statistics for entire dataset

HR_sumStats = pd.DataFrame(df_HR[["satisfaction_level",
                                  "last_evaluation",
                                  "number_project",
                                  "average_montly_hours",
                                  "time_spend_company"]].mean())

HR_sumStats

,0
satisfaction_level,0.612834
last_evaluation,0.716102
number_project,3.803054
average_montly_hours,201.050337
time_spend_company,3.498233


In [ ]:
# Descriptive Statistics by Stayers vs Leavers
## values_count() for Work_accident, promotion_last_5years, Departments

HR_byStayLeave = df_HR.groupby('left').mean()
# HR_sumStats_byStayLeave = pd.DataFrame(HR_byStayLeave[["satisfaction_level", "last_evaluation",
#                                                    "number_project","average_montly_hours",
#                                                     "time_spend_company"]]).aggregate([np.mean, np.median, np.var, np.std].round(2))

# HR_sumStats_byStayLeave

HR_byStayLeave

ANALYSIS

In [ ]:
# Research Question: 1) Why do valuable employees leave